<a href="https://colab.research.google.com/github/sergiogf93/MetNumerics/blob/master/notebooks/08_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressió lineal

Per què utilitzar la regressió lineal?

1. Fàcil d'utilitzar
2. Fàcil d'interpretar
3. Base molts mètodes
4. Computacionalment ràpida.
5. Molta gent n'ha sentit a parlar :)

### Llibreries de Python per la regressió lineal

Les més populars són

1. `scikit-learn`
2. `statsmodels`


## Problema

Podem predir el preu de la marihuana en un estat a partir de l'informació demogràfica?

Trobarem, pel gener de 2015, la mitjana del preu de la marihuana d'alta qualitat en tots els estats. Suposem que no coneixem quins són els preus en els següents estats:

<br>
iowa, kentucky, missouri, nevada, wyoming, south dakota, new jersey, michigan, idaho
<br>

Aquest serà el nostre dataset **test**. La resta d'estats són el nostre dataset **train**. Necessitem entrenar el nostre model al train dataset i fer prediccions sobre el test dataset.

Com que de fet sí coneixem els preus pels estats al test dataset, podem verificar quant bo són els nostres models.

In [0]:
import pandas as pd

In [0]:
#Load the data
weed_pd = pd.read_csv("https://raw.githubusercontent.com/sergiogf93/MetNumerics/master/data/Weed_Price.csv", parse_dates=[-1])
demo_pd = pd.read_csv("https://raw.githubusercontent.com/sergiogf93/MetNumerics/master/data/Demographics_State.csv")

In [0]:
weed_pd.head()

In [0]:
weed_pd.dtypes

In [0]:
demo_pd.head()

In [0]:
demo_pd.dtypes

Veiem que el dataset de preus té els noms dels estats amb la lletra inicial en majúscula. Podem utilitzar el següent command per convertir-la en minúscula.

In [0]:
str.lower("Alabama")

In [0]:
?weed_pd.apply

In [0]:
weed_pd.State = weed_pd["State"].apply(lambda x: str.lower(x))

In [0]:
weed_pd.head()

Podem mirar quins estats hi ha als dataset:

In [0]:
pd.unique(weed_pd.State)

In [0]:
pd.unique(demo_pd.region)

In [0]:
# Prenem el mes i l'any a partir de `date` per poder sel·leccionar gener 2015
weed_pd["Month"] = weed_pd["date"].apply(lambda x: x.month)
weed_pd["Year"] = weed_pd["date"].apply(lambda x: x.year)

In [0]:
weed_jan2015_pd = weed_pd.loc[(weed_pd.Year==2015) & (weed_pd.Month==1)]

In [0]:
weed_jan2015_pd.head()

Agrupem les dades en funció de l'estat i fem la mitjana dels valors de HighQ:

In [0]:
weed_jan2015_summarized = weed_jan2015_pd[["State", "HighQ"]].groupby("State").mean().reset_index()

In [0]:
#The source price dataset for our model 
weed_jan2015_summarized

Definim el conjunt test:

In [0]:
test_states = ["iowa", "kentucky", "missouri", "nevada", "wyoming", \
               "south dakota", "new jersey", "michigan", "idaho" ]

Agrupem el dataset que conté els preus per estat amb el que conté la informació demogràfica. 

In [0]:
data_for_model = pd.merge(weed_jan2015_summarized, demo_pd, left_on="State", right_on="region")

In [0]:
data_for_model.head()

Creem un array de booleans per distingir quines files corresponen al test dataset i quines al train dataset:

In [0]:
#Now, creating train and test dataset
criterion = weed_jan2015_summarized["State"].map(lambda x: x in test_states)
#Another way to do it
#criterion = weed_jan2015_summarized.State.isin(test_states)

In [0]:
print "Train data labels: \n", ~criterion, "\n\n", "Test data labels: \n", criterion

In [0]:
train = data_for_model[~criterion]
test = data_for_model[criterion]

In [0]:
train.shape, test.shape

In [0]:
train

In [0]:
test

### Regressió lineal

Utilitzarem `statsmodels`

La regressió lineal és de la forma:

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$


On els $\beta$ són els *coeficients del model*.

Els coeficients del model són estimats a partir de les dades segons diferents mètodes.

**Mètode dels mínims quadrats**: Identifiquem els pesos de manera que la solució minimitzi la suma quadrada dels erros per cada conjunt de dades.

[Wiki](https://en.wikipedia.org/wiki/Least_squares)



In [0]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

In [0]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

Comencem per visualitzar les dades. Fes una gràfica dels punts (x,y) on x són els valors de la columna 'percent_white' i y són els 'HighQ' pel dataset train. Fes-ne un altre amb x = total_population i un últim amb x = per_capita_income

In [0]:
# Fes un plot HighQ vs percent_white

# Fes un plot HighQ vs total_population

# Fes un plot HighQ vs per_capita_income

In [0]:
#@title
sns.regplot(x="percent_white", y = "HighQ", data=train)

In [0]:
#@title
sns.regplot(x="total_population", y="HighQ", data=train)

In [0]:
#@title
sns.regplot(x="per_capita_income", y="HighQ", data=train)

**Fer fits per un altre plot/model**

In [0]:
?sns.lmplot

In [0]:
sns.lmplot(x="total_population", y="HighQ", data=train, order=3)

**Plot de correlacions**

In [0]:
#More plots.

#Visualizing correlation matrix using a heatmap

sns.heatmap(train.corr())

**Múltiples plots**

In [0]:
?sns.pairplot

In [0]:
#Multiple scatter plot

sns.pairplot(train, x_vars='HighQ', y_vars=['total_population', 'percent_white', 'percent_black', \
                                           'per_capita_income', 'median_rent', 'median_age'], kind='reg')

**Primer, fem un model d'una variable**

Intentem estimar el preu en funció de la població

In [0]:
feature_columns = ["total_population"]
train_x = train[feature_columns]
train_y = train['HighQ']

In [0]:
model_1 = LinearRegression()
model_1.fit(train_x, train_y)

In [0]:
print model_1.intercept_
print model_1.coef_

**Fem servir el model per fer prediccions**

In [0]:
model_1_predict = model_1.predict(test[feature_columns])

In [0]:
model_1_predict = pd.DataFrame({'States': test.State, 'Actual Price': test.HighQ, 'Predicted Price': model_1_predict})

In [0]:
model_1_predict

**Calculem l'Error quadràtic mig**

In [0]:
#Root Mean square error on test dataset
np.sqrt(np.mean(np.square(model_1_predict['Actual Price'] - \
                         model_1_predict['Predicted Price'])))

**Com més petit és l'EQM, millor el model**

**Exercici** Fes un model utilitzant com a variables `total_population` i `per_capita_income`. Digues quin és el EQM obtingut.

In [0]:
# Fes un model de regressió lineal amb x = [total_population, per_capita_income] i y = HighQ

# Digues quin és l'EQM obtingut.

In [0]:
#@title
feature_columns = ["total_population","per_capita_income"]
train_x = train[feature_columns]
train_y = train['HighQ']

model_2 = LinearRegression()
model_2.fit(train_x, train_y)

print model_2.intercept_
print model_2.coef_

model_2_predict = model_2.predict(test[feature_columns])

model_2_predict = pd.DataFrame({'States': test.State, 'Actual Price': test.HighQ, 'Predicted Price': model_2_predict})

np.sqrt(np.mean(np.square(model_2_predict['Actual Price'] - \
                         model_2_predict['Predicted Price'])))